In [1]:
import pandas as pd
import numpy as np

In [2]:
Parkhäuser = pd.read_csv("../Final_Data/Cleaned/1000014cleaned.csv")
Flugbewegungen = pd.read_csv("../Final_Data/Cleaned/Airport_traffic_hourly_cleaned.csv")
Gasverbrauch = pd.read_csv("../Final_Data/Cleaned/Gasverbrauch_cleaned.csv")
#Schallmessungen = pd.read_csv("../Final_Data/Cleaned/Schallmessungen_cleaned.csv", sep=";") -> zu viele Datenpunkte fehlen
Stromverbrauch = pd.read_csv("../Final_Data/Cleaned/Stromverbrauch_Basel_hourly_cleaned.csv")
Wetterdaten = pd.read_csv("../Final_Data/Cleaned/Wetterdaten_Basel_cleaned22_24.csv")
Verkehrsdaten = pd.read_csv("../Final_Data/Cleaned/1000006cleaned.csv")

In [3]:

Parkhäuser.rename(columns={"RoundedTime": "Datum"}, inplace=True)
Parkhäuser.head()
#Flugbewegungen.drop(columns=["Unnamed: 0"], inplace=True)
Flugbewegungen.head()
#Gasverbrauch.head()
#Schallmessungen.drop(columns=["Unnamed: 0"], inplace=True)
#Schallmessungen.head()
#Stromverbrauch.drop(columns=["Unnamed: 0"], inplace=True)
#Stromverbrauch.head()
#Wetterdaten.head()
#Verkehrsdaten.rename(columns={"DateTimeFrom": "Datum"}, inplace=True)
#Verkehrsdaten.head()




,Unnamed: 0,Datum,Hour,Traffic
0,0,2022-01-01 00:00:00+00:00,0,1.0
1,1,2022-01-01 01:00:00+00:00,1,0.0
2,2,2022-01-01 02:00:00+00:00,2,0.0
3,3,2022-01-01 03:00:00+00:00,3,0.0
4,4,2022-01-01 04:00:00+00:00,4,0.0


In [4]:
Parkhäuser.head()

,Datum,Rebgassechange,Clarahuuschange,Citychange,Storchenchange,Post Baselchange,Aeschenchange,Bahnhof Südchange,Bad. Bahnhofchange,Europechange,Claramattechange,Elisabethenchange,Steinenchange,Kunstmuseumchange,Messechange,Anfoschange,Centralbahnparkingchange
0,2022-01-01 00:00:00+00:00,-7.0,-6.0,-15.0,-19.0,0.0,-2.0,-2.0,-3.0,-1.0,-23.0,-5.0,-10.0,-4.0,-7.0,0.0,-2.0
1,2022-01-01 01:00:00+00:00,-7.0,-1.0,-4.0,-9.0,-1.0,0.0,0.0,0.0,0.0,0.0,-9.0,-19.0,0.0,-8.0,-1.0,-1.0
2,2022-01-01 02:00:00+00:00,-4.0,0.0,-1.0,-1.0,0.0,0.0,-1.0,0.0,0.0,0.0,-12.0,-23.0,0.0,-8.0,0.0,0.0
3,2022-01-01 03:00:00+00:00,-3.0,0.0,-1.0,-6.0,0.0,0.0,0.0,0.0,0.0,0.0,-15.0,-10.0,0.0,-2.0,0.0,0.0
4,2022-01-01 04:00:00+00:00,-2.0,0.0,2.0,2.0,0.0,0.0,2.0,0.0,1.0,0.0,-1.0,-7.0,0.0,-2.0,0.0,1.0


from datetime import timedelta



Flugbewegungen["Datum"] = pd.to_datetime(Flugbewegungen["Datum"]) + Flugbewegungen["Hour"].apply(lambda x: timedelta(hours=x))
Flugbewegungen.head()

Flugbewegungen["Datum"] = Flugbewegungen["Datum"].astype(object)
Flugbewegungen["Datum"].dtype

In [5]:
Verkehrsdaten = Verkehrsdaten.rename(columns={"DateTimeFrom": "Datum"})
print(Verkehrsdaten['Datum'].isna().sum())


0


In [6]:
#standardise the data
from sklearn.preprocessing import StandardScaler
import numpy as np
import pickle

dfs = [Parkhäuser, Flugbewegungen, Gasverbrauch, Stromverbrauch, Wetterdaten, Verkehrsdaten]

# Dictionary to store the mean and standard deviation for each dataframe
scalers = {}

# Loop through each dataframe
for i, df in enumerate(dfs):
    numeric_cols = df.select_dtypes(include=[np.number])
    if not numeric_cols.empty:
        if i == 1:  # Exclude 'Hour' column for Flugbewegungen
            numeric_cols = numeric_cols.drop(columns=['Hour'])
        print(f"Processing df{i} with columns: {numeric_cols.columns}")
        scaler = StandardScaler()
        # Fit the scaler on the dataframe and transform the data
        scaled_data = scaler.fit_transform(numeric_cols)
        
        # Save the mean and standard deviation
        scalers[f'df{i}'] = {'mean': scaler.mean_, 'scale': scaler.scale_}
        
        # Replace the original data with the standardized data
        df[numeric_cols.columns] = scaled_data
        dfs[i] = df
    else:
        print(f"df{i} has no numeric columns or is empty.")

# Assign the transformed dataframes back to their original names
Parkhäuser, Flugbewegungen, Gasverbrauch, Stromverbrauch, Wetterdaten, Verkehrsdaten = dfs

# Save the scalers dictionary for later use
with open('scalers.pkl', 'wb') as f:
    pickle.dump(scalers, f)


Processing df0 with columns: Index(['Rebgassechange', 'Clarahuuschange', 'Citychange', 'Storchenchange',
       'Post Baselchange', 'Aeschenchange', 'Bahnhof Südchange',
       'Bad. Bahnhofchange', 'Europechange', 'Claramattechange',
       'Elisabethenchange', 'Steinenchange', 'Kunstmuseumchange',
       'Messechange', 'Anfoschange', 'Centralbahnparkingchange'],
      dtype='object')
Processing df1 with columns: Index(['Unnamed: 0', 'Traffic'], dtype='object')
Processing df2 with columns: Index(['Gasverbrauch'], dtype='object')
Processing df3 with columns: Index(['Stromverbrauch'], dtype='object')
Processing df4 with columns: Index(['Unnamed: 0', 'Basel Temperature [2 m elevation corrected]',
       'Basel Precipitation Total', 'Basel Wind Speed [10 m]',
       'Basel Wind Direction [10 m]'],
      dtype='object')
Processing df5 with columns: Index(['350n_sumPW', '350n_sumLief', '350n_sumLW', '350v_sumPW',
       '350v_sumLief', '350v_sumLW', '352n_sumPW', '352n_sumLief',
       '352

In [7]:
print(type(Verkehrsdaten))
Wetterdaten.drop(columns=["Unnamed: 0"], inplace=True)
Flugbewegungen.drop(columns=["Unnamed: 0"], inplace=True)
Gasverbrauch.sort_values(by=["Datum"], inplace=True)

<class 'pandas.core.frame.DataFrame'>


In [8]:

Flugbewegungen["Datum"].dtype

dtype('O')

In [9]:
                                                                                                # The merge isnt correct because it erases the Hour format and therefore setting the entire Day to one value?!
# Merge all dataframes on the 'Datum' column
final_df = Parkhäuser.merge(Wetterdaten, on='Datum', how='inner') \
                      .merge(Stromverbrauch, on='Datum', how='inner') \
                      .merge(Verkehrsdaten, on='Datum', how='inner') \
                      .merge(Gasverbrauch, on='Datum', how='inner') \
                      .merge(Flugbewegungen, on='Datum', how='inner') \

#                      .merge(Schallmessungen, on='Datum', how='inner') \ -> too many NaN values
# Display the first few rows of the merged dataframe
final_df.head(20)

,Datum,Rebgassechange,Clarahuuschange,Citychange,Storchenchange,Post Baselchange,Aeschenchange,Bahnhof Südchange,Bad. Bahnhofchange,Europechange,...,84111104v_sumLW,84111108n_sumPW,84111108n_sumLief,84111108n_sumLW,84111108v_sumPW,84111108v_sumLief,84111108v_sumLW,Gasverbrauch,Hour,Traffic
0,2022-01-01 00:00:00+00:00,-0.489808,-1.162325,-0.163786,-1.110112,-0.000054,-0.258093,-0.245491,-0.158272,-0.151270,...,-0.117866,-0.685184,-0.566797,-1.055921,-0.813705,-0.413276,-1.072953,0.294494,0,-1.142151
1,2022-01-01 01:00:00+00:00,-0.489808,-0.193898,-0.043944,-0.525891,-0.191296,-0.000173,-0.000190,-0.000462,-0.000203,...,-0.746090,-0.954499,-0.775801,-1.055921,-1.047891,-0.767192,-1.072953,0.357518,1,-1.295479
2,2022-01-01 02:00:00+00:00,-0.279837,-0.000213,-0.011259,-0.058515,-0.000054,-0.000173,-0.122840,-0.000462,-0.000203,...,-0.746090,-0.936545,-0.566797,-1.055921,-1.056899,-0.413276,-1.072953,0.385414,2,-1.295479
3,2022-01-01 03:00:00+00:00,-0.209846,-0.000213,-0.011259,-0.350625,-0.000054,-0.000173,-0.000190,-0.000462,-0.000203,...,-0.746090,-1.044270,-0.566797,-1.055921,-1.201013,-0.413276,-1.072953,0.617131,3,-1.295479
4,2022-01-01 04:00:00+00:00,-0.139856,-0.000213,0.021425,0.116751,-0.000054,-0.000173,0.245112,-0.000462,0.150864,...,-0.327274,-1.062225,-0.775801,-1.055921,-1.255056,-0.649220,-1.072953,1.109860,4,-1.295479
5,2022-01-01 05:00:00+00:00,0.140106,-0.000213,0.925692,-0.233781,-0.000054,0.128787,-0.000190,0.052141,-0.000203,...,-0.536682,-1.035293,-0.775801,-1.055921,-1.146970,-0.531248,-1.072953,1.670331,5,-1.295479
6,2022-01-01 06:00:00+00:00,-0.069865,0.193473,-0.022154,-0.116937,-0.000054,0.128787,-0.122840,-0.000462,-0.151270,...,-0.536682,-1.160973,-0.566797,0.002343,-1.255056,-0.767192,-1.072953,1.455102,6,-0.682167
7,2022-01-01 07:00:00+00:00,0.000125,-0.193898,0.054109,-0.058515,-0.000054,-0.000173,0.122461,0.052141,-0.151270,...,-0.327274,-1.089156,-0.775801,-1.055921,-1.210020,-0.767192,-1.072953,1.472270,7,-0.222182
8,2022-01-01 08:00:00+00:00,0.000125,0.193473,0.141268,-0.000093,-0.000054,-0.000173,0.613065,-0.053065,-0.151270,...,-0.746090,-0.999385,-0.671299,-0.526789,-1.047891,-0.649220,-0.611437,1.555734,8,-0.835495
9,2022-01-01 09:00:00+00:00,-0.069865,1.161899,0.173952,-0.175359,0.573671,-0.000173,0.613065,0.104744,-0.151270,...,-0.327274,-0.918590,-0.253292,-0.526789,-0.975834,-0.531248,-0.611437,1.401258,9,-0.528839


In [10]:
print(final_df.tail(25))

                           Datum  Rebgassechange  Clarahuuschange  Citychange  \
24549  2024-10-19 21:00:00+00:00       -0.979741        -0.000213   -0.632262   
24550  2024-10-19 22:00:00+00:00       -1.399684        -0.000213   -0.250945   
24551  2024-10-19 23:00:00+00:00       -1.119722        -0.968639   -0.109312   
24552  2024-10-20 00:00:00+00:00       -0.139856        -0.000213   -0.076628   
24553  2024-10-20 01:00:00+00:00        0.000125        -0.000213   -0.022154   
24554  2024-10-20 02:00:00+00:00        0.000125        -0.000213    0.010530   
24555  2024-10-20 03:00:00+00:00        0.000125        -0.000213   -0.000365   
24556  2024-10-20 04:00:00+00:00        0.070116         0.968214    0.860323   
24557  2024-10-20 05:00:00+00:00        0.000125        -0.000213   -0.022154   
24558  2024-10-20 06:00:00+00:00        0.140106        -0.000213    0.032320   
24559  2024-10-20 07:00:00+00:00        0.560049        -0.000213    0.250215   
24560  2024-10-20 08:00:00+0

In [11]:
test_dataFrame = [Parkhäuser,
                Wetterdaten,
                Flugbewegungen,
                Gasverbrauch,
                Stromverbrauch,
                Verkehrsdaten]

for i in test_dataFrame:
    print(i['Datum'].max())

2024-10-21 14:00:00+00:00
2024-11-22 23:00:00+00:00
2024-11-22 23:00:00+00:00
2024-11-24 04:00:00+00:00
2024-11-20 22:00:00+00:00
2024-10-20 21:00:00+00:00


# Check for rows with NaN values
nan_rows = final_df[final_df.isnull().any(axis=1)]

# Display the rows with NaN values
nan_rows.head()

In [12]:
type(final_df[["Datum"]])

pandas.core.frame.DataFrame

In [13]:
final_df.to_csv("../Working_DataFrame.csv", index=False)


#final_df.tail(40)
final_df['Datum'].dtype
import numpy as np

cutoff = np.datetime64('2024-11-20 10:00:00')
final_df = final_df[final_df['Datum'] <= cutoff]
final_df.tail(40)

In [14]:
Flugbewegungen.head()

,Datum,Hour,Traffic
0,2022-01-01 00:00:00+00:00,0,-1.142151
1,2022-01-01 01:00:00+00:00,1,-1.295479
2,2022-01-01 02:00:00+00:00,2,-1.295479
3,2022-01-01 03:00:00+00:00,3,-1.295479
4,2022-01-01 04:00:00+00:00,4,-1.295479


In [15]:

Stromverbrauch["Datum"].dtype
Wetterdaten["Datum"].dtype
Gasverbrauch["Datum"].dtype

Flugbewegungen["Datum"] = Flugbewegungen["Datum"].astype(str)
Flugbewegungen["Datum"].dtype

dtype('O')

In [16]:
#DF NN
DF_NN = Gasverbrauch.merge(Stromverbrauch, on='Datum', how='left') \
                      .merge(Wetterdaten, on='Datum', how='left') \
                      .merge(Flugbewegungen, on='Datum', how='left')

In [17]:
DF_NN.head()

# Convert the date column to datetime format
DF_NN['Datum'] = pd.to_datetime(DF_NN['Datum'])

# Filter the dataframe to keep only dates before the specified date and time
DF_NN = DF_NN[DF_NN['Datum'] <= '2024-10-27 13:00']
DF_NN.head()


,Gasverbrauch,Datum,Stromverbrauch,Basel Temperature [2 m elevation corrected],Basel Precipitation Total,Basel Wind Speed [10 m],Basel Wind Direction [10 m],Hour,Traffic
0,0.294494,2022-01-01 00:00:00+00:00,-1.116259,-0.926467,-0.314805,-0.694066,0.115891,0.0,-1.142151
1,0.357518,2022-01-01 01:00:00+00:00,-1.368317,-1.041967,-0.314805,-0.918024,0.047361,1.0,-1.295479
2,0.385414,2022-01-01 02:00:00+00:00,-1.523703,-1.162435,-0.314805,-0.857675,-0.143112,2.0,-1.295479
3,0.617131,2022-01-01 03:00:00+00:00,-1.546518,-1.135112,-0.314805,-0.831118,-0.006355,3.0,-1.295479
4,1.109860,2022-01-01 04:00:00+00:00,-1.432685,-1.163677,-0.314805,-0.687750,-0.148316,4.0,-1.295479


In [18]:

DF_NN.drop(columns=["Datum", "Hour"], inplace=True)

DF_NN.to_csv("./Working_DataFrame_NN.csv", index=False)

In [19]:
#2 NaN in Stromverbrauch 2022-10-30 01:00:00 & 2023-10-29 01:00:00

# Verkehrsdatenset hat 5 NaN values 2024-10-20 22:00:00 & 2024-10-20 23:00:00 & 2024-10-21 00:00:00 & 2024-10-21 01:00:00 & 2024-10-21 02:00:00	

# Wetter hat 5 NaN values 2023-12-22 00:00:00 & 2023-12-22 01:00:00 & 2023-12-22 02:00:00 & 2023-12-22 03:00:00 & 2023-12-22 04:00:00
# habe nochmal nachgeschaut und es ist der ganze 22 und 23 Dezember 2023 || ist aber nur eine Auflösung die ich ausgewählt habe, sollte mit der anderen Auflösung nicht mehr vorkommen


In [20]:
final_df.head()

,Datum,Rebgassechange,Clarahuuschange,Citychange,Storchenchange,Post Baselchange,Aeschenchange,Bahnhof Südchange,Bad. Bahnhofchange,Europechange,...,84111104v_sumLW,84111108n_sumPW,84111108n_sumLief,84111108n_sumLW,84111108v_sumPW,84111108v_sumLief,84111108v_sumLW,Gasverbrauch,Hour,Traffic
0,2022-01-01 00:00:00+00:00,-0.489808,-1.162325,-0.163786,-1.110112,-0.000054,-0.258093,-0.245491,-0.158272,-0.151270,...,-0.117866,-0.685184,-0.566797,-1.055921,-0.813705,-0.413276,-1.072953,0.294494,0,-1.142151
1,2022-01-01 01:00:00+00:00,-0.489808,-0.193898,-0.043944,-0.525891,-0.191296,-0.000173,-0.000190,-0.000462,-0.000203,...,-0.746090,-0.954499,-0.775801,-1.055921,-1.047891,-0.767192,-1.072953,0.357518,1,-1.295479
2,2022-01-01 02:00:00+00:00,-0.279837,-0.000213,-0.011259,-0.058515,-0.000054,-0.000173,-0.122840,-0.000462,-0.000203,...,-0.746090,-0.936545,-0.566797,-1.055921,-1.056899,-0.413276,-1.072953,0.385414,2,-1.295479
3,2022-01-01 03:00:00+00:00,-0.209846,-0.000213,-0.011259,-0.350625,-0.000054,-0.000173,-0.000190,-0.000462,-0.000203,...,-0.746090,-1.044270,-0.566797,-1.055921,-1.201013,-0.413276,-1.072953,0.617131,3,-1.295479
4,2022-01-01 04:00:00+00:00,-0.139856,-0.000213,0.021425,0.116751,-0.000054,-0.000173,0.245112,-0.000462,0.150864,...,-0.327274,-1.062225,-0.775801,-1.055921,-1.255056,-0.649220,-1.072953,1.109860,4,-1.295479


In [21]:
import pandas as pd
df = final_df.copy()
# Ensure 'Datum' is a datetime object
df['Datum'] = pd.to_datetime(df['Datum'])

# Create a complete range of hourly timestamps from the first to the last point
full_range = pd.date_range(start=df['Datum'].min(), end=df['Datum'].max(), freq='h')

# Identify missing timestamps
missing_timestamps = full_range.difference(df['Datum'])

# Display the results
if missing_timestamps.empty:
    print("No missing hourly timestamps. All hours are accounted for.")
else:
    print("Missing hourly timestamps:")
    print(missing_timestamps)

No missing hourly timestamps. All hours are accounted for.
